In [227]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [228]:
import warnings
from analytics.summarize import ReviewVisualizer
from analytics.analyzer import ReviewAnalyzer  
from analytics.connector import OracleReviewLoader
from analytics.path_config import BASE_DIR

In [229]:
#suppress all warnings
warnings.filterwarnings("ignore")

In [230]:
loader = OracleReviewLoader(user="myappuser", password="My$ecurePassw0rd", dsn="localhost/XEPDB1")

try:
    loader.connect()
    df = loader.fetch_reviews()
finally:
    loader.disconnect()

[2025-06-10 16:26:19,684] INFO in connector: Successfully connected to Oracle database.
[2025-06-10 16:26:21,050] INFO in connector: Fetched 2350 review records from Oracle DB.
[2025-06-10 16:26:21,056] INFO in connector: Oracle database connection closed.


In [231]:
df.sample(10)

,bank_name,review_text,rating,sentiment_label,keywords,themes
870,CBE,wow . what i can say thank you.,5.0,positive,"[""wow"", ""thank""]","[""Miscellaneous""]"
1046,BOA,there is no speed,1.0,positive,"[""speed""]","[""Miscellaneous""]"
12,CBE,"Very bad, can't even load they need to improve their services.",1.0,negative,"[""need improve"", ""not load"", ""improve"", ""load"", ""service""]","[""Miscellaneous""]"
1858,Dashen,best app,5.0,positive,"[""good app"", ""good"", ""app""]","[""Miscellaneous""]"
964,BOA,best banking app in the wworld,5.0,positive,"[""good banking"", ""banking app"", ""banking"", ""good"", ""app""]","[""Miscellaneous""]"
1967,Dashen,I love this app,5.0,positive,"[""love"", ""app""]","[""Miscellaneous""]"
887,BOA,good,5.0,positive,"[""good""]","[""Miscellaneous""]"
2108,Dashen,Wow,5.0,positive,"[""wow""]","[""Miscellaneous""]"
1759,BOA,Good,5.0,positive,"[""good""]","[""Miscellaneous""]"
1340,BOA,The worst app,1.0,negative,"[""bad app"", ""bad"", ""app""]","[""Miscellaneous""]"


In [232]:
df.rename(columns={
    'review_text': 'REVIEW_TEXT',
    'rating': 'RATING',
    'sentiment_label': 'SENTIMENT_LABEL',
    'themes': 'THEMES'
}, inplace=True)


In [233]:
analyzer=ReviewAnalyzer(df)

[2025-06-10 16:26:21,229] INFO in analyzer: Initializing analyzer with 2350 reviews.


In [234]:
# _ = analyzer.analyze_per_bank(top_n=3, display=True)  # discard return value, avoid raw print


In [235]:
summary_df=analyzer.generate_summary_df()

[2025-06-10 16:26:21,362] INFO in analyzer: Generated summary DataFrame with 41 rows.


In [236]:
rv=ReviewVisualizer(summary_df,df,out_dir=BASE_DIR/'reports')
rv.generate_all_visuals_per_bank();

[2025-06-10 16:26:21,411] INFO: Visualizer initialized with output path: /home/teshager/Documents/10Academy/repositories/projects/customer-experience-analytics-for-fintech-apps/reports
[2025-06-10 16:26:21,845] INFO: Saved bar chart: /home/teshager/Documents/10Academy/repositories/projects/customer-experience-analytics-for-fintech-apps/reports/BOA_positive_bar.png
[2025-06-10 16:26:22,870] INFO: Saved word cloud: /home/teshager/Documents/10Academy/repositories/projects/customer-experience-analytics-for-fintech-apps/reports/BOA_positive_wordcloud.png
[2025-06-10 16:26:23,304] INFO: Saved bar chart: /home/teshager/Documents/10Academy/repositories/projects/customer-experience-analytics-for-fintech-apps/reports/BOA_negative_bar.png
[2025-06-10 16:26:24,427] INFO: Saved word cloud: /home/teshager/Documents/10Academy/repositories/projects/customer-experience-analytics-for-fintech-apps/reports/BOA_negative_wordcloud.png
[2025-06-10 16:26:24,807] INFO: Saved bar chart: /home/teshager/Documents

In [ ]:
rv.export_visuals_to_pdf()

In [ ]:
rv.export_visuals_and_insights_to_pdf()
rv.analyzer = analyzer  # Connect them
# Run insights
insights = analyzer.analyze_per_bank()
rv.insights = insights
# Export all
rv.recommendations =[rv.insights]
rv.export_visuals_and_insights_to_pdf("Fintech_Bank_Review_Report.pdf")

[2025-06-10 16:25:59,294] INFO: Exported combined visuals and insights report: /home/teshager/Documents/10Academy/repositories/projects/customer-experience-analytics-for-fintech-apps/reports/Review_Report_with_Insights.pdf
[2025-06-10 16:25:59,357] INFO in analyzer: Generated insights for 3 banks.


                      ✨ Insights for CBE                      
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃     Top Drivers      ┃ Count ┃   Top Pain Points    ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│  ["miscellaneous"]   │   658 │  ["miscellaneous"]   │   222 │
├──────────────────────┼───────┼──────────────────────┼───────┤
│ ["feature requests"] │    10 │ ["feature requests"] │    26 │
├──────────────────────┼───────┼──────────────────────┼───────┤
│   ["reliability"]    │     1 │   ["reliability"]    │     9 │
└──────────────────────┴───────┴──────────────────────┴───────┘

                         ✨ Insights for BOA                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃         Top Drivers         ┃ Count ┃   Top Pain Points    ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│      ["miscellaneous"]      │   443 │  ["miscellaneous"]   │   420 │
├─────────────────────────────┼───────┼──────────────────────┼───────┤
│    ["feature requests"]     │    12 │ ["feature requests"] │    32 │
├─────────────────────────────┼───────┼──────────────────────┼───────┤
│ ["transaction performance"] │     2 │   ["reliability"]    │    21 │
└─────────────────────────────┴───────┴──────────────────────┴───────┘

                        ✨ Insights for Dashen                        
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃     Top Drivers      ┃ Count ┃       Top Pain Points       ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│  ["miscellaneous"]   │   312 │      ["miscellaneous"]      │    93 │
├──────────────────────┼───────┼─────────────────────────────┼───────┤
│ ["feature requests"] │    14 │ ["transaction performance"] │     6 │
├──────────────────────┼───────┼─────────────────────────────┼───────┤
│ ["user experience"]  │     7 │    ["feature requests"]     │     3 │
└──────────────────────┴───────┴─────────────────────────────┴───────┘

[2025-06-10 16:26:09,510] INFO: Exported combined visuals and insights report: /home/teshager/Documents/10Academy/repositories/projects/customer-experience-analytics-for-fintech-apps/reports/Fintech_Bank_Review_Report.pdf


In [ ]:
df= df.dropna(subset=['bank_name', 'THEMES'])

In [ ]:
df_copy=df.copy()

In [ ]:
df_boa=df[df['bank_name']=='BOA']
df_cbe=df[df['bank_name']=='CBE']
df_dashen=df[df['bank_name']=='Dashen']

In [ ]:
df_copy

,bank_name,REVIEW_TEXT,RATING,SENTIMENT_LABEL,keywords,THEMES
0,CBE,it is sooo good,5.0,positive,"[""good""]","[""Miscellaneous""]"
1,CBE,excellent,5.0,positive,"[""excellent""]","[""Miscellaneous""]"
2,CBE,ok,1.0,positive,"[""ok""]","[""Miscellaneous""]"
3,CBE,Fantastic,5.0,positive,"[""fantastic""]","[""Miscellaneous""]"
4,CBE,accessible to using,5.0,positive,"[""accessible""]","[""Miscellaneous""]"
...,...,...,...,...,...,...
2345,BOA,"Worst App it ain't allowing yo activate not even in the nearest branch are able to activate it. I have been client of the bank for 10years. What is this BoA? As we know it being a leading in simplifying banking service this App needs to be fixed. Then we will give it better rate,",1.0,negative,"[""activate"", ""banking service"", ""client"", ""near"", ""lead""]","[""Miscellaneous""]"
2346,BOA,እኔ ከሀገር ውጪ ነው ያለሁት I need help please!!! የበፊቱ app አልከፍት ብሎኝ አብዴት ማድረግ አልቻልኩም ከ ፕሌይ ስቶር ላይ እንደ አዲስ ነው ዳውንሎድ የሚጠይቀው እሱንም ሳደርግ በ ኢትዮጵያ ስልክ ቁጥር ነው ኮድ የሚልከው ፣የኢትዮጵያ ቅጥሬ አሁን active አይደለም ምን ማድረግ እችላለው? በምን መንገድ ልትረዱኝ ትችላላችሁ?,1.0,negative,"[""\u1290\u12cd"", ""\u120b\u12ed"", ""\u12a0\u1201\u1295"", ""active"", ""help""]","[""Miscellaneous""]"
2347,BOA,"The app does not work well from abroad, there have been continuous upgrades and updates that are very interruptive and non-responsive. Staff do not have the technical capabilities to support whereas the support number and online support are never available.",1.0,negative,"[""support"", ""staff"", ""abroad"", ""available"", ""online""]","[""Customer Support""]"
2348,BOA,App good. how money transfer tele birr agents,1.0,positive,"[""tele birr"", ""tele"", ""app good"", ""birr"", ""money""]","[""Miscellaneous""]"


## Bank Of Abysinia

In [ ]:
reviews_df=df_boa[(df['THEMES']== '["Feature Requests"]')][['REVIEW_TEXT','SENTIMENT_LABEL','RATING']]
reviews_df.style.set_properties(**{'text-align': 'right'}, subset=['REVIEW_TEXT'])
reviews_df

,REVIEW_TEXT,SENTIMENT_LABEL,RATING
872,"Hello, I’m facing a problem with the BOA Mobile app. Every time I enter my phone number and password, the app crashes and shows an error that says “BoaMobile closed because this app has a bug.” I tried updating, reinstalling, and clearing cache, but nothing worked. Please fix this bug in the next update. I really need access to my account. Thank you.",negative,1.0
876,This App is not interest for Android phone Please update it .,negative,1.0
975,Always do update and that is annoying,negative,3.0
1025,it's not work correctly... you must have update it,positive,1.0
1028,good but they don't update enough don't add new things,negative,3.0
1069,It has good features but sometimes it doesn't work...0,positive,1.0
1161,The latest update has stopped android 9 phones not to open this App. Never work properly since i started using it.,negative,1.0
1174,"The worst app update ever. Why would you include the ""developer option off"" thing here? Why do you care? It is my phone my money, right? That is the reason why I switched to other banks. So either turn it off or everyone will discontinue their business one by one.",negative,1.0
1233,It's not working. It needs a big update. Can't you update the app quickly?,negative,1.0
1400,After update it doesn't work well,negative,2.0


In [ ]:
pd.set_option('display.max_colwidth', None)
update_rows=reviews_df[bool].sample(7)
update_rows


,REVIEW_TEXT,SENTIMENT_LABEL,RATING
1616,It was working good before the last update but after the update it asks to disable developer option. why is that? That means any developer with developer option enabled can't access his account. It crushs a lot of times.,negative,2.0
1689,I can't update the enable button not open.,negative,1.0
1417,After recent update the app keeps asking to disable developers options and won't let you use it unless you do.,positive,2.0
1560,አሁን update የተደረገው App ምንም ነው የማይሰራው መከራችንን እኮ ነው ያሳየን😏😏),negative,1.0
1422,App is any updates why?,positive,5.0
1161,The latest update has stopped android 9 phones not to open this App. Never work properly since i started using it.,negative,1.0
1659,"With every update the app keeps on getting worse,it is filled with bugs",negative,1.0


In [ ]:
# Negative comments related to updates
neg_update_reviews=update_rows[(update_rows['SENTIMENT_LABEL']=='negative') |(update_rows['RATING']<3)].sort_values(by='RATING')
neg_update_reviews

,REVIEW_TEXT,SENTIMENT_LABEL,RATING
1689,I can't update the enable button not open.,negative,1.0
1560,አሁን update የተደረገው App ምንም ነው የማይሰራው መከራችንን እኮ ነው ያሳየን😏😏),negative,1.0
1659,"With every update the app keeps on getting worse,it is filled with bugs",negative,1.0
1161,The latest update has stopped android 9 phones not to open this App. Never work properly since i started using it.,negative,1.0
1417,After recent update the app keeps asking to disable developers options and won't let you use it unless you do.,positive,2.0
1616,It was working good before the last update but after the update it asks to disable developer option. why is that? That means any developer with developer option enabled can't access his account. It crushs a lot of times.,negative,2.0


In [ ]:
df_boa['keywords']
df_boa['new keywords'] = df_boa['keywords'].str.replace(r'\[|\] |]', '', regex=True)
df_boa

,bank_name,REVIEW_TEXT,RATING,SENTIMENT_LABEL,keywords,THEMES,Cleaned_Text_Regex,new keywords
871,BOA,it's not working,3.0,positive,"[""work""]","[""Miscellaneous""]",it's not working,"""work"""
872,BOA,"Hello, I’m facing a problem with the BOA Mobile app. Every time I enter my phone number and password, the app crashes and shows an error that says “BoaMobile closed because this app has a bug.” I tried updating, reinstalling, and clearing cache, but nothing worked. Please fix this bug in the next update. I really need access to my account. Thank you.",1.0,negative,"[""bug"", ""update"", ""show error"", ""cache"", ""hello""]","[""Feature Requests""]","Hello, I’m facing a problem with the BOA Mobile app. Every time I enter my phone number and password, the app crashes and shows an error that says “BoaMobile closed because this app has a bug.” I tried updating, reinstalling, and clearing cache, but nothing worked. Please fix this bug in the next update. I really need access to my account. Thank you.","""bug"", ""update"", ""show error"", ""cache"", ""hello"""
873,BOA,exceptional,5.0,positive,"[""exceptional""]","[""Miscellaneous""]",exceptional,"""exceptional"""
874,BOA,BoA Mobile good bank,5.0,positive,"[""good bank"", ""boa mobile"", ""boa"", ""mobile"", ""bank""]","[""Miscellaneous""]",BoA Mobile good bank,"""good bank"", ""boa mobile"", ""boa"", ""mobile"", ""bank"""
875,BOA,this is worest app 24/7 loading,1.0,negative,"[""loading"", ""wor app"", ""wor"", ""app""]","[""Miscellaneous""]",this is worest app 24/7 loading,"""loading"", ""wor app"", ""wor"", ""app"""
...,...,...,...,...,...,...,...,...
2345,BOA,"Worst App it ain't allowing yo activate not even in the nearest branch are able to activate it. I have been client of the bank for 10years. What is this BoA? As we know it being a leading in simplifying banking service this App needs to be fixed. Then we will give it better rate,",1.0,negative,"[""activate"", ""banking service"", ""client"", ""near"", ""lead""]","[""Miscellaneous""]","Worst App it ain't allowing yo activate not even in the nearest branch are able to activate it. I have been client of the bank for 10years. What is this BoA? As we know it being a leading in simplifying banking service this App needs to be fixed. Then we will give it better rate,","""activate"", ""banking service"", ""client"", ""near"", ""lead"""
2346,BOA,እኔ ከሀገር ውጪ ነው ያለሁት I need help please!!! የበፊቱ app አልከፍት ብሎኝ አብዴት ማድረግ አልቻልኩም ከ ፕሌይ ስቶር ላይ እንደ አዲስ ነው ዳውንሎድ የሚጠይቀው እሱንም ሳደርግ በ ኢትዮጵያ ስልክ ቁጥር ነው ኮድ የሚልከው ፣የኢትዮጵያ ቅጥሬ አሁን active አይደለም ምን ማድረግ እችላለው? በምን መንገድ ልትረዱኝ ትችላላችሁ?,1.0,negative,"[""\u1290\u12cd"", ""\u120b\u12ed"", ""\u12a0\u1201\u1295"", ""active"", ""help""]","[""Miscellaneous""]",እኔ ከሀገር ውጪ ነው ያለሁት I need help please!!! የበፊቱ app አልከፍት ብሎኝ አብዴት ማድረግ አልቻልኩም ከ ፕሌይ ስቶር ላይ እንደ አዲስ ነው ዳውንሎድ የሚጠይቀው እሱንም ሳደርግ በ ኢትዮጵያ ስልክ ቁጥር ነው ኮድ የሚልከው ፣የኢትዮጵያ ቅጥሬ አሁን active አይደለም ምን ማድረግ እችላለው? በምን መንገድ ልትረዱኝ ትችላላችሁ?,"""\u1290\u12cd"", ""\u120b\u12ed"", ""\u12a0\u1201\u1295"", ""active"", ""help"""
2347,BOA,"The app does not work well from abroad, there have been continuous upgrades and updates that are very interruptive and non-responsive. Staff do not have the technical capabilities to support whereas the support number and online support are never available.",1.0,negative,"[""support"", ""staff"", ""abroad"", ""available"", ""online""]","[""Customer Support""]","The app does not work well from abroad, there have been continuous upgrades and updates that are very interruptive and non-responsive. Staff do not have the technical capabilities to support whereas the support number and online support are never available.","""support"", ""staff"", ""abroad"", ""available"", ""online"""
2348,BOA,App good. how money transfer tele birr agents,1.0,positive,"[""tele birr"", ""tele"", ""app good"", ""birr"", ""money""]","[""Miscellaneous""]",App good. how money transfer tele birr agents,"""tele birr"", ""tele"", ""app good"", ""birr"", ""money"""
